<a href="https://colab.research.google.com/github/lonerpatti/BigDataPos/blob/main/Uni_Facef_Pyspark_Atividade_Nota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nova seção

Essa atividade tem como objetivo descobrir um pouco mais sobre o cliente e traçar um perfil do mesmo. Visto que temos as informações pessoais do cliemte em um arquivo chamado "cliente_fake.csv", e os dados transacionais desse cliente no arquivo chamado "venda_fake". E idéia do exercicio é a manipulação dos dados de forma a fazer agregações, join nos dados, podendo ser necessário o uso de UDFs, gerando um Dataframe Final:

#### O Dataframe final deve estar agrupado por cliente (cliente_id), e conter as seguintes informações:

- cliente_id - O identificador do cliente
- idade - Calcular a idade do cliente (valor inteiro)
- recencia - Calcular a quantos meses faz que o cliente não compra (valor inteiro)
- qt_total_compras - Quantidade total de compras (pedidos distintos)
- vr_total_compras = Valor total de compras do cliente
- qt_total_itens - Quantidade total de itens comprados pelo cliente (produto_id distintos)
- qt_max_parcelas - Quantidade máxima de parcelas que o cliente já utilizou
- perfil_cliente - Deve mostrar o perfil de compra desse cliente, se compra somente "loja" ou somente "online", ou em ambos "multicanal";

OBS: O dataset "venda_fake" está a nível de itens. Pode haver mais de um item por pedido, isto é, o numero do pedido pode estar duplicado nodataset.


# <font color='blue'>Uni-Facef - Pyspark - Atividade</font>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving cliente_fake.csv to cliente_fake.csv
Saving venda_fake.csv to venda_fake.csv


In [3]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 42.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=efde85e7814af8a5f86691586ec67524c4bc9162097343048a862241f038fa4c
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [5]:
# coding: utf-8
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [6]:
dir(f)

['Column',
 'DataFrame',
 'DataType',
 'PandasUDFType',
 'PythonEvalType',
 'SparkContext',
 'StringType',
 'UserDefinedFunction',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_binary_mathfunctions',
 '_collect_list_doc',
 '_collect_set_doc',
 '_create_binary_mathfunction',
 '_create_column_from_literal',
 '_create_column_from_name',
 '_create_function',
 '_create_function_over_column',
 '_create_udf',
 '_create_window_function',
 '_functions',
 '_functions_1_4_over_column',
 '_functions_1_6_over_column',
 '_functions_2_1_over_column',
 '_functions_2_4',
 '_functions_deprecated',
 '_functions_over_column',
 '_lit_doc',
 '_message',
 '_options_to_str',
 '_string_functions',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_window_functions',
 '_wrap_deprecated_function',
 'abs',
 'acos',
 'add_months',
 'approxCountDistinct',
 'approx_count_distinct',
 'array',
 'array_contains',
 'array_distinct',
 'array_exc

In [7]:
spark = SparkSession.builder \
    .appName('ConhecendoCliente') \
    .getOrCreate()

type(spark)

pyspark.sql.session.SparkSession

#### Dataset de cliente

In [8]:
df_cliente = spark.read \
    .option("delimiter", "|") \
    .csv('cliente_fake.csv', header=True)

df_cliente.show(50)

+--------------+----------+-------------+
|          nome|cliente_id|dt_nascimento|
+--------------+----------+-------------+
|Cliente 100013|    100013|   1964-06-21|
|Cliente 100014|    100014|   1964-05-24|
|Cliente 100015|    100015|   1946-10-18|
|Cliente 100016|    100016|   1972-03-24|
|Cliente 100017|    100017|   1982-01-19|
|Cliente 100018|    100018|   1983-04-26|
|Cliente 100019|    100019|   1971-09-09|
|Cliente 100020|    100020|   1989-06-18|
|Cliente 100007|    100007|   1976-11-04|
|Cliente 100008|    100008|   1980-12-01|
|Cliente 100009|    100009|   1981-03-05|
|Cliente 100010|    100010|   1984-03-20|
|Cliente 100011|    100011|   1977-11-22|
|Cliente 100012|    100012|   1974-02-04|
|Cliente 100001|    100001|   2009-04-16|
|Cliente 100002|    100002|   1974-08-29|
|Cliente 100003|    100003|   1981-12-05|
|Cliente 100004|    100004|   1968-08-08|
|Cliente 100005|    100005|   1982-11-29|
|Cliente 100006|    100006|   1948-04-01|
+--------------+----------+-------

In [9]:
df_cliente.printSchema()

root
 |-- nome: string (nullable = true)
 |-- cliente_id: string (nullable = true)
 |-- dt_nascimento: string (nullable = true)



#### Dataset de pedido

In [10]:
df_venda = spark.read \
    .option("delimiter", "|") \
    .csv('venda_fake.csv', header=True)

df_venda.show(50)

+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|     pedido_id|cliente_id|           dt_pedido|produto_id|qt_parcelas|   forma_pagto|vr_unit_item|qt_item|tp_canal|
+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|10000720200217|    100007|2020-02-17T00:00:...|   6253644|         20|cartao credito|         478|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   5313408|         20|cartao credito|      1357.2|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   6253140|         20|cartao credito|         372|      1|    loja|
|10000720200625|    100007|2020-06-25T00:00:...|   6858780|          7|cartao credito|       317.8|      1|    site|
|10000120200626|    100001|2020-06-26T00:00:...|   7560984|         16|cartao credito|     1399.04|      1|    loja|
|10000920200614|    100009|2020-06-14T00:00:...|   7571280|     

In [11]:
df_venda.printSchema()

root
 |-- pedido_id: string (nullable = true)
 |-- cliente_id: string (nullable = true)
 |-- dt_pedido: string (nullable = true)
 |-- produto_id: string (nullable = true)
 |-- qt_parcelas: string (nullable = true)
 |-- forma_pagto: string (nullable = true)
 |-- vr_unit_item: string (nullable = true)
 |-- qt_item: string (nullable = true)
 |-- tp_canal: string (nullable = true)



In [13]:
# Criando a view de Cliente
df_cliente.createOrReplaceTempView("v_cliente")

In [15]:
# Criando a view de do Df Vendas p/ visualizar pedidos
df_venda.createOrReplaceTempView("v_pedido")

# **Dataframe utilizando apenas sparkSQL**

In [19]:
df_cliente_perfil = spark.sql(
    """
    select v_cliente.cliente_id, 
           cast((datediff(current_date(), v_cliente.dt_nascimento) / 365) as int) as idade,
           cast((datediff(current_date(), max(v_pedido.dt_pedido)) / 30) as int) as recencia,
           count(distinct v_pedido.pedido_id) as qt_total_compras,
           round(sum(v_pedido.vr_unit_item * v_pedido.qt_item ),2) as vr_total_compras,
           count(distinct v_pedido.produto_id) as qt_total_itens,
           max(v_pedido.qt_parcelas) as qt_max_parcelas,
           case count(distinct v_pedido.tp_canal)
            when 1 then min(v_pedido.tp_canal)
            else "multicanal"
           end as perfil_cliente
           /*max(v_pedido.dt_pedido) as ultima_compra, count(distinct v_pedido.tp_canal) as qt_canal,*/
      from v_pedido
        inner join v_cliente on v_pedido.cliente_id = v_cliente.cliente_id
      group by v_cliente.cliente_id,
        cast((datediff(current_date(), v_cliente.dt_nascimento) / 365) as int)
      order by cast(v_cliente.cliente_id as int)
    """)

df_cliente_perfil.show(100)

+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|cliente_id|idade|recencia|qt_total_compras|vr_total_compras|qt_total_itens|qt_max_parcelas|perfil_cliente|
+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|    100001|   11|       4|               1|         1399.04|             1|             16|          loja|
|    100002|   46|       8|               1|         1280.07|             1|              9|          loja|
|    100003|   38|       2|               2|         1904.95|             3|              9|          loja|
|    100004|   52|      10|               2|         6860.08|             2|             24|    multicanal|
|    100005|   37|       8|               2|         3207.28|             2|             15|          loja|
|    100006|   72|       2|               1|         3170.72|             3|              9|          loja|
|    100007|   44|       4| 

# **Resultado da junção de clientes e vendas**

In [20]:
df_resultado = df_venda.join(df_cliente,'cliente_id','inner') \
             .withColumn("multiplica", sf.col("vr_unit_item") * sf.col("qt_item")) \
             .withColumn("dt_pedido", sf.col("dt_pedido").cast("date")) \
             .withColumn("qt_parcelas", sf.col("qt_parcelas").cast("int"))

df_resultado.show()  

+----------+--------------+----------+----------+-----------+--------------+------------+-------+--------+--------------+-------------+----------+
|cliente_id|     pedido_id| dt_pedido|produto_id|qt_parcelas|   forma_pagto|vr_unit_item|qt_item|tp_canal|          nome|dt_nascimento|multiplica|
+----------+--------------+----------+----------+-----------+--------------+------------+-------+--------+--------------+-------------+----------+
|    100007|10000720200217|2020-02-17|   6253644|         20|cartao credito|         478|      1|    loja|Cliente 100007|   1976-11-04|     478.0|
|    100007|10000720200217|2020-02-17|   5313408|         20|cartao credito|      1357.2|      1|    loja|Cliente 100007|   1976-11-04|    1357.2|
|    100007|10000720200217|2020-02-17|   6253140|         20|cartao credito|         372|      1|    loja|Cliente 100007|   1976-11-04|     372.0|
|    100007|10000720200625|2020-06-25|   6858780|          7|cartao credito|       317.8|      1|    site|Cliente 1000

# **Com o perfil do CLiente e Vendas  agrego cliente, sumarizando suas informações**

In [25]:
df_agrega_cliente = df_resultado.groupBy("cliente_id")\
                 .agg(sf.max("dt_pedido").alias("max_dt_pedido"),
                      sf.countDistinct("pedido_id").alias("qt_total_compras"),
                      sf.countDistinct("tp_canal").alias("qt_canal"),
                      sf.countDistinct("produto_id").alias("qt_total_itens"),
                      sf.max("dt_nascimento").alias("dt_nascimento"),
                      sf.min("tp_canal").alias("min_tp_canal"),
                      sf.max("qt_parcelas").alias("qt_max_parcelas"))\
.withColumn("recencia",(sf.months_between(sf.current_date(),sf.col("max_dt_pedido").cast("date"))))\
.withColumn("recencia", sf.col("recencia").cast("int")) \
.withColumn("idade",(sf.datediff(sf.current_date(),sf.col("dt_nascimento").cast("date")))/365.0)\
.withColumn("idade",sf.col("idade").cast("int"))\
.withColumn("perfil_cliente", sf.when(sf.col("qt_canal") == 1, sf.col("min_tp_canal"))
            .otherwise("multicanal"))
               
df_agrega_cliente.show()

+----------+-------------+----------------+--------+--------------+-------------+------------+---------------+--------+-----+--------------+
|cliente_id|max_dt_pedido|qt_total_compras|qt_canal|qt_total_itens|dt_nascimento|min_tp_canal|qt_max_parcelas|recencia|idade|perfil_cliente|
+----------+-------------+----------------+--------+--------------+-------------+------------+---------------+--------+-----+--------------+
|    100010|   2020-01-29|               1|       1|             1|   1984-03-20|        loja|              9|       9|   36|          loja|
|    100014|   2020-06-18|               2|       1|             3|   1964-05-24|        loja|             11|       4|   56|          loja|
|    100008|   2020-07-12|               1|       1|             1|   1980-12-01|        loja|             14|       3|   39|          loja|
|    100006|   2020-08-10|               1|       1|             3|   1948-04-01|        loja|              9|       2|   72|          loja|
|    100005| 

# **Do mesmo modo, a agregação de pedido totalizando cliente/pedido**

In [26]:
df_agrega_pedido  = df_resultado.groupBy("cliente_id") \
               .agg({"multiplica": "sum"}) \
               .withColumnRenamed("sum(multiplica)", "vr_total_compras") \
               .withColumn("vr_total_compras", sf.round("vr_total_compras", 2))
               
   
df_agrega_pedido.show() 

+----------+----------------+
|cliente_id|vr_total_compras|
+----------+----------------+
|    100010|          898.97|
|    100014|         1746.01|
|    100008|         1399.86|
|    100006|         3170.72|
|    100005|         3207.28|
|    100007|          2525.0|
|    100020|          755.66|
|    100009|          4478.0|
|    100004|         6860.08|
|    100015|         2592.99|
|    100016|          181.08|
|    100003|         1904.95|
|    100019|         6114.96|
|    100011|          1324.8|
|    100012|         1099.98|
|    100013|         7053.58|
|    100017|         4176.88|
|    100002|         1280.07|
|    100018|         1437.96|
|    100001|         1399.04|
+----------+----------------+



In [27]:
df_agrega_cliente.createOrReplaceTempView("v_agrega_cliente")

In [28]:
df_agrega_pedido.createOrReplaceTempView("v_agrega_pedido")

# **Resultado agregando DataFrames e SparkSQL**


In [29]:
df_final = spark.sql(
    """
    select v_agrega_cliente.cliente_id,
           idade, 
           recencia,
           qt_total_compras,           
           v_agrega_pedido.vr_total_compras,
           qt_total_itens,
           qt_max_parcelas,
           perfil_cliente
      from v_agrega_pedido
        inner join v_agrega_cliente on v_agrega_pedido.cliente_id = v_agrega_cliente.cliente_id
      order by cast(v_agrega_cliente.cliente_id as int)
    """)


df_final.show()

+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|cliente_id|idade|recencia|qt_total_compras|vr_total_compras|qt_total_itens|qt_max_parcelas|perfil_cliente|
+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|    100001|   11|       4|               1|         1399.04|             1|             16|          loja|
|    100002|   46|       8|               1|         1280.07|             1|              9|          loja|
|    100003|   38|       2|               2|         1904.95|             3|             10|          loja|
|    100004|   52|       9|               2|         6860.08|             2|             24|    multicanal|
|    100005|   37|       8|               2|         3207.28|             2|             15|          loja|
|    100006|   72|       2|               1|         3170.72|             3|              9|          loja|
|    100007|   44|       4| 

Missão cumprida! Até a próxima.